In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd
plt.rcParams['figure.figsize'] = (20, 6)
import numpy as np

# data utils
from mmk.data import make_root_db, Database

# debug, interact
from mmk.utils import show, audio

# segmenting function
from mmk.extract.segment import from_recurrence_matrix

### Stretch Function

In [ ]:
from scipy.interpolate import RectBivariateSpline as RBS

def stretch(S, ratio):
    if S.shape[1] <= 1 or S.shape[0] <= 1:
        return S
    time_indices = np.linspace(0, S.shape[1]-1, int(np.rint(S.shape[1]/ratio)))
    mag, phase = S, None
    spline = RBS(np.arange(S.shape[0]), np.arange(S.shape[1]), mag)
    interp = spline.ev(np.arange(S.shape[0])[:, None], time_indices)
    return interp


### Data

In [ ]:
# where you have audio files 
my_music_folder = "gould/"

# the file with your data :
my_db = "gould.h5"

if not os.path.exists(my_db):
    make_root_db(my_db, my_music_folder)

db = Database(my_db)

db.metadata

### Select and Segment a Piece

In [ ]:
piece = db.metadata.iloc[[0]]

X = db.fft.get(piece).T

segments = from_recurrence_matrix(X,
                                 L=6,
                                 k=None,
                                 sym=True,
                                 bandwidth=1.,
                                 thresh=0.2,
                                 min_dur=4)
segments.describe()

### Quantize segments' durations to some multiple

In [ ]:
def nearest_multiple(x, m):
    """
    quantize a sequence x to a given modulo m
    """
    return (m * np.maximum(np.rint(x / m), 1)).astype(np.int)

durs = segments.durations_
# use the mean duration as multiple
mean_dur = int(durs.mean())

q_durs = nearest_multiple(durs, mean_dur)

### Stretch segments according to their new durations

In [ ]:
new = []
for i, seg_slice in enumerate(segments.slices(time_axis=1)):
    x = X[seg_slice]
    new += [stretch(x, (durs[i] / q_durs[i]))]
    
new = np.concatenate(new, axis=1)

show(new)
audio(new)